In [13]:
import sys
import os
import pandas as pd
import numpy as np
to_add = os.getcwd().split('/scripts')[0]
if not to_add in sys.path:
    sys.path.insert(0, to_add)

import ferretfit.aws as aws

In [ ]:
def ferret_fit(folder, prefix, download=True, bucket_name='ccurtis.data'):

    filename = '{}.csv'.format(prefix)
    if download:
        aws.download_s3('{}/{}'.format(folder, filename), filename,
                        bucket_name=bucket_name)
    ferret_data = pd.read_csv(filename)
    ferret_data = ferret_data.sort_values(by=['X'])
    length = ferret_data.shape[0]

    x = ferret_data['X']
    y = ferret_data['Y']
    fine_t, data_fit, eamp, efr = fit_sine(x, y, gfr=1/100)

    lowess = sm.nonparametric.lowess
    ymid = lowess(y, x, frac=0.3)
    yavg = np.convolve(y, np.ones((length,))/length, mode='same')

    strait = np.mean(y)*np.ones((length,))
    intersections = intersection(x, ymid[:, 1], x, strait)
    pawcount = len(x)
    pawdens = np.abs(100*pawcount/(max(x) - min(x)))
    stride = np.mean(np.diff(x))
    stridestd = np.std(np.diff(x))

    plt.figure(figsize=(10, 5))
    plt.scatter(x, y, s=300)
    plt.plot(x, ymid[:, 1], linewidth=6)
    plt.plot(x, strait, 'k', linewidth=6)
    plt.plot(fine_t, data_fit, 'm', linewidth=6)
    # plt.plot(x, yavg, 'k', linewidth=6)
    plt.ylim(0, 120)
    imfile = '{}_fit.png'.format(prefix)
    plt.savefig(imfile)
    aws.upload_s3(imfile, '{}/{}'.format(folder, imfile),
                  bucket_name='ccurtis.data')

    ystd = np.round(np.std(y), 2)
    yrange = np.round(np.ptp(y), 2)
    rsd = 100*np.round(ystd/np.mean(y), 2)
    cross = len(intersections[0])
    crossdens = np.abs(100*cross/(max(x) - min(x)))

    ffparams = Bunch(ystd=ystd, yrange=yrange, rsd=rsd, amp=np.abs(eamp),
                     period=1/efr, pawcount=pawcount, pawdens=pawdens,
                     cross=cross, crossdens=crossdens, stride=stride,
                     stridestd=stridestd)

    return ffparams

In [7]:
prefix = 'F10_T1_R1'
folder = 'ferret_tracking/09_12_18_all_data'
bucket_name = 'ccurtis.data'

In [14]:
feet = ['LH', 'RH', 'LF', 'RF']
ystdfoot = {}
yrangefoot = {}
yrsdfoot = {}

download = True
for foot in feet:
    filename = '{}_{}.csv'.format(prefix, foot)
    if download:
        aws.download_s3('{}/{}'.format(folder, filename), filename,
                        bucket_name=bucket_name)
    ferret_data = pd.read_csv(filename)
    ferret_data = ferret_data.sort_values(by=['X'])
    #if feet in ['LH', 'RH']:
    #    if ferret_data.shape([0]) < 4:
    #        ferret_data = ferret_data
    #    else:
    ferret_data = ferret_data[ferret_data['Area'] > 199]
    #else:
    #    ferret_data = ferret_data

    try:
        y = ferret_data['Y']
        ystdfoot[foot] = np.round(np.std(y), 2)
        yrangefoot[foot] = np.round(np.ptp(y), 2)
        yrsdfoot[foot] = 100*np.round(ystdfoot[foot]/np.mean(y), 2)
    except:
        ystdfoot[foot] = np.nan
        yrangefoot[foot] = np.nan
        yrsdfoot[foot] = np.nan
    
#ffparams.ystdfoot = ystdfoot
#ffparams.yrangefoot = yrangefoot
#ffparams.yrsdfoot = yrsdfoot

In [11]:
ferret_data

,,Area,Mean,Min,Max,X,Y,XM,YM
0,1,108,0,0,0,206,80.500,206,80.500
2,3,86,0,0,0,227,91.384,227,91.384
1,2,86,0,0,0,410,89.384,410,89.384


In [15]:
ystdfoot

{'LF': 11.390000000000001, 'LH': nan, 'RF': 7.7599999999999998, 'RH': 0.0}

In [16]:
yrangefoot

{'LF': 27.489999999999998, 'LH': nan, 'RF': 18.07, 'RH': 0.0}